<a href="https://colab.research.google.com/github/GenevaKirwan/CFG-Airline-Project/blob/Test-1/Processing_Fleet_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysing Fleet Portfolio as it Related to Emmissions

In [1]:
## Introduction to Section

In [2]:
## Retrieving the Raw Data

In [3]:
### Initial Data Source

In [4]:
### Revised Data Source

In [5]:
### ICAO Type Designator API

In [6]:
## Data Joining and Cleansing

In [7]:
## Data Visualisation

In [8]:
## Analysis

# Retrieving and Cleansing Fleet Data

The aim of this code is to web-scrape fleet data tables from https://data.freshaviation.co.uk/airlines/ and cleanse the data to minimise the size of the resultant dataframe. The data source details every plane owned by each airline, however what I need is the number of each aircraft type so I can compare this 'fleet portfolio' with the emmisions data.

www.airfleets.net prohibits bots

## Web Scraping Precautions

I checked the data source website for a robots.txt file to check for any bot limitations before starting but there was no file present. Regardless, when scraping the data I will still include a 10 secs waiting time before retrieving data to make sure I don't overload the site.

## Initial tests and familiarisation with webscraping and BeautifulSoup

I ran the quick check below with requests to see if I was successfully retrieving the page from the site.

In [9]:
import requests
url = 'https://data.freshaviation.co.uk/view-airline/American%20Airlines/'

r = requests.get(url)

print(r.content[:100])

b'<!DOCTYPE html>\r\n<html lang="en">\r\n\r\n<head>\r\n\t<meta charset="utf-8">\r\n\t<meta name="viewport" content'


I also practised with BeautifulSoup to retrieve information from the right part of the relevant table. This took quite a bit of tweaking as it was my first time using CSS selector and BeautifulSoup syntax. Within this code I am pulling the name and IDs for the airlines. this will be useful in stitching my data with the data produced by the rest of the team (if necessary)

In [10]:
from bs4 import BeautifulSoup
import csv

soup = BeautifulSoup(r.content, 'html.parser')

IDs = soup.select('table tr td')[3].text.split(' / ')
Name = soup.select('table tr td')[1].text
ICAO = IDs[0]
IATA = IDs [1]
print('ICAO: ' + ICAO + '\n' + 'IATA: ' + IATA)

ICAO: AAL
IATA: AA


## Final Code and Walkthrough

The page URLS are generated into the URLs we will need to call from in the code below. 

In [11]:
airlines = ['American Airlines', 'Southwest Airlines', 'Delta Air Lines', 'United Airlines', 'Spirit Airlines', 'Alaska Airlines', 'JetBlue Airways', 'SkyWest Airlines', 'Frontier Airlines', 'Hawaiian Airlines']
urls = []

for airline in airlines:
    urls.append('https://data.freshaviation.co.uk/view-airline/' + airline.replace(' ','%20') + '/')
    
urls

['https://data.freshaviation.co.uk/view-airline/American%20Airlines/',
 'https://data.freshaviation.co.uk/view-airline/Southwest%20Airlines/',
 'https://data.freshaviation.co.uk/view-airline/Delta%20Air%20Lines/',
 'https://data.freshaviation.co.uk/view-airline/United%20Airlines/',
 'https://data.freshaviation.co.uk/view-airline/Spirit%20Airlines/',
 'https://data.freshaviation.co.uk/view-airline/Alaska%20Airlines/',
 'https://data.freshaviation.co.uk/view-airline/JetBlue%20Airways/',
 'https://data.freshaviation.co.uk/view-airline/SkyWest%20Airlines/',
 'https://data.freshaviation.co.uk/view-airline/Frontier%20Airlines/',
 'https://data.freshaviation.co.uk/view-airline/Hawaiian%20Airlines/']

Next I am going to use a for loop to web-scrape the airline fleet data for each of the URLs. This for-loop will be quite long as by the end of it I will have placed all the data I need from all the site-pages into a few lists. Comments will be placed within the code to provide further insight into the steps carried out. WARNING:This code block may take up to 10 mins to run.

In [12]:
import requests
from time import sleep
from bs4 import BeautifulSoup

#Some lists I will need later to collate the info into
airline_name = []
ICAO_ID = []
IATA_ID = []
aircraft_type_col = []
no_aircrafts = []

for url in urls:
    #First we use requests to get the HTML from the inserted url.
    r = requests.get(url)
    
    #Next we use BeautifulSoup to locate the Name and IDs of the airline from the HTML.
    soup = BeautifulSoup(r.content, 'html.parser')
    IDs = soup.select('table tr td')[3].text.split(' / ')
    Name = soup.select('table tr td')[1].text
    ICAO = IDs[0]
    IATA = IDs [1]
    
    #Then we collect the fleet specific table from the HTML.
    sleep(10)
    rows = soup.select('#cfleet table tr')
    
    #As I only need the 'Aircraft Type' from this table I will collect this value in a list.
    all_aircraft_list = []
    for row in rows:
        r_data = row.select('td')
        Aircraft_Type = r_data[2].text
        all_aircraft_list.append(Aircraft_Type)
    
    #My list has many duplicate values. I want to know how many times each Aircraft type is listed.
    #To achieve this I will create a dictionary holding the count of each list item.
    all_aircraft_dict = dict.fromkeys(all_aircraft_list)
    for key in all_aircraft_dict:
        all_aircraft_dict[key] = all_aircraft_list.count(key)
        
    #Now I would like to append these values to lists ready to be converted later into a pandas dataframe.
    aircraft_type = list(all_aircraft_dict.keys())
    airline_name.extend([Name]*len(aircraft_type))
    ICAO_ID.extend([ICAO]*len(aircraft_type))
    IATA_ID.extend([IATA]*len(aircraft_type))
    aircraft_type_col.extend(aircraft_type)
    no_aircrafts.extend(list(all_aircraft_dict.values()))
    
    #Lastly I will have the code wait 10 secs before pulling from the next page
    sleep(10)
    
print('All done :-)')

All done :-)


All of the lists created now need to be moved into a pandas dataframe ready for further processing later. It's also a handy way for me to be able to visualise the data I have produced and 'sanity check' it. The dataframe is produced by using the pd zip command to combine all the lists, this way they are in the right format for the pd.DataFrame function.  

In [13]:
import pandas as pd
fleet_pd = pd.DataFrame(list(zip(airline_name, ICAO_ID, IATA_ID, aircraft_type_col, no_aircrafts)), 
                           columns =['Airline Name', 'ICAO ID', 'IATA ID', 'Aircraft Model', 'No Aircrafts'])
fleet_pd

,Airline Name,ICAO ID,IATA ID,Aircraft Model,No Aircrafts
0,American Airlines,AAL,AA,Airbus A321-200,218
1,American Airlines,AAL,AA,Airbus A320-200,48
2,American Airlines,AAL,AA,Airbus A319-100,133
3,American Airlines,AAL,AA,Boeing B757-200,30
4,American Airlines,AAL,AA,Boeing B737-800,304
...,...,...,...,...,...
89,Frontier Airlines,FFT,F9,Airbus A319-100,6
90,Hawaiian Airlines,HAL,HA,Airbus A321neo,18
91,Hawaiian Airlines,HAL,HA,Airbus A330-200,24
92,Hawaiian Airlines,HAL,HA,Boeing B717-200,19


I will then save the data as a csv to protect it from changes/ loss of this code or changes to the websites data is being scraped from.

In [14]:
from datetime import date
today = date.today()
file_name = 'fleetcsv_' + today.strftime("%d%m%y") + '.csv'
fleet_pd.to_csv(file_name)

My table is missing one crucial piece of data: the engine type. Whilst it is useful to know specific aircraft makes, the emmissions analysis would be enhanced grouping these airplanes based on their type. This can be achieved using the ICAO Aircraft Type Designators API. However, the input to get the relevant table is the manufacturer name. The following code will create a list of aircraft manufacturers from the Aircraft Model names.

In [15]:
import re
man_ls = []

for model in fleet_pd['Aircraft Model']:
    mod_words = re.split(' |-',model)
    first_word = mod_words[0].upper()
    if first_word.upper() not in man_ls:
        man_ls.append(first_word)
print(man_ls)


['AIRBUS', 'BOEING', 'MCDONNELL', 'EMBRAER', 'BOMBARDIER']


Now I will use the ICAO API to retrieve the table of aircraft types for these manufacturers

In [16]:
api_key = '40f6d960-e4c7-41d9-ad8d-8b0b0030ae77'
url = 'https://applications.icao.int/dataservices/api/type-list?'
type_des_db = pd.DataFrame(columns = ['manufacturer_code', 'model_no', 'engine_count', 'engine_type', 'tdesig'])

for man in man_ls:
    r = requests.get(url + 'api_key=' + api_key+ '&format=csv&manufacturer=' + man)
    data = r.content.decode('utf-8').splitlines()
    
    with open("tmp.csv", "w") as csv_file:
        writer = csv.writer(csv_file, delimiter = '\t')
        for line in data:
            writer.writerow(re.split('\s+',line))
            
    temp_db = pd.read_csv('tmp.csv')
    temp_db.drop(labels = ['model_name', 'model_version', 'aircraft_desc', 'description', 'wtc', 'wtg'], axis=1, inplace =True)
    type_des_db = type_des_db.append(temp_db)
    
type_des_db

,manufacturer_code,model_no,engine_count,engine_type,tdesig
0,AIRBUS,A-350-900\tXWB,2,Jet,A359
1,AIRBUS,A-350-900\tXWB\tPrestige,2,Jet,A359
2,AIRBUS,Prestige\t(A-350-900),2,Jet,A359
3,AIRBUS,C-212,2,Turboprop/Turboshaft,C212
4,AIRBUS,A-300B2,2,Jet,A30B
...,...,...,...,...,...
27,BOMBARDIER,BD-700,2,Jet,GLEX
28,BOMBARDIER,BD-700,2,Jet,GL7T
29,BOMBARDIER,NaN,2,Jet,GL5T
30,BOMBARDIER,NaN,2,Jet,GLEX


In [17]:
type_des_db.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
type_des_db.dropna(axis=0, how='any', thresh=None, subset= ['model_no'], inplace=True)
type_des_db

,manufacturer_code,model_no,engine_count,engine_type,tdesig
0,AIRBUS,A-350-900\tXWB,2,Jet,A359
1,AIRBUS,A-350-900\tXWB\tPrestige,2,Jet,A359
2,AIRBUS,Prestige\t(A-350-900),2,Jet,A359
3,AIRBUS,C-212,2,Turboprop/Turboshaft,C212
4,AIRBUS,A-300B2,2,Jet,A30B
...,...,...,...,...,...
21,BOMBARDIER,CSeries\tCS300,2,Jet,BCS3
22,BOMBARDIER,CS100,2,Jet,BCS1
23,BOMBARDIER,CS300,2,Jet,BCS3
24,BOMBARDIER,CL-600\t,2,Jet,CRJ7


In [18]:
file_name = 'type_des_' + today.strftime("%d%m%y") + '.csv'
type_des_db.to_csv(file_name)

In [19]:
inputString = 'ggib gbuu6jhn 600 kbkbk'

mod_ls = inputString.split(' ')
ref_mod_ls = []
for word in mod_ls:
    if bool(re.search(r'\d', word)) == True:
        ref_mod_ls.append(word)

ref_mod_ls

['gbuu6jhn', '600']

In [20]:
str = 'Boeing beatyy A300-900'

def tdesig_maker (make):
    
    mod_ls = make.split(' ')
    ref_mod_ls = []
    for word in mod_ls:
        if bool(re.search(r'\d', word)) == True:
            ref_mod_ls.append(word)
    critical_wrd = ref_mod_ls[0]

    if '-' in critical_wrd:
        broken_mod_ls = critical_wrd.split('-')
        if  '0' in broken_mod_ls[0]:
            tdesig = broken_mod_ls[0][0:3] + broken_mod_ls[1][0]
        else:
            lead_mod = broken_mod_ls[0][0:4]
            tdesig = lead_mod + broken_mod_ls[1][0:(4-len(lead_mod))]
    else:
        tdesig = critical_wrd[0:4]
    return tdesig

tdesig_maker(str)

'A309'

adding the tdesig column

In [21]:
fleet_pd['tdesig'] = fleet_pd.apply(lambda row: tdesig_maker(row['Aircraft Model']),axis=1)
fleet_pd

,Airline Name,ICAO ID,IATA ID,Aircraft Model,No Aircrafts,tdesig
0,American Airlines,AAL,AA,Airbus A321-200,218,A321
1,American Airlines,AAL,AA,Airbus A320-200,48,A322
2,American Airlines,AAL,AA,Airbus A319-100,133,A319
3,American Airlines,AAL,AA,Boeing B757-200,30,B757
4,American Airlines,AAL,AA,Boeing B737-800,304,B737
...,...,...,...,...,...,...
89,Frontier Airlines,FFT,F9,Airbus A319-100,6,A319
90,Hawaiian Airlines,HAL,HA,Airbus A321neo,18,A321
91,Hawaiian Airlines,HAL,HA,Airbus A330-200,24,A332
92,Hawaiian Airlines,HAL,HA,Boeing B717-200,19,B717


In [22]:
all_fleet_pd = pd.merge(
    fleet_pd,
    type_des_db,
    how="left",
    on=None,
    left_on= 'tdesig',
    right_on= 'tdesig',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)
all_fleet_pd

,Airline Name,ICAO ID,IATA ID,Aircraft Model,No Aircrafts,tdesig,manufacturer_code,model_no,engine_count,engine_type
0,American Airlines,AAL,AA,Airbus A319-100,133,A319,AIRBUS,A-319,2,Jet
1,American Airlines,AAL,AA,Airbus A319-100,133,A319,AIRBUS,VC-1\tACJ,2,Jet
2,American Airlines,AAL,AA,Airbus A319-100,133,A319,AIRBUS,ACJ\t(A-319),2,Jet
3,Delta Air Lines,DAL,DL,Airbus A319-100,57,A319,AIRBUS,A-319,2,Jet
4,Delta Air Lines,DAL,DL,Airbus A319-100,57,A319,AIRBUS,VC-1\tACJ,2,Jet
...,...,...,...,...,...,...,...,...,...,...
166,JetBlue Airways,JBU,B6,Embraer EMB-190,60,EMB1,NaN,NaN,NaN,NaN
167,American Airlines,AAL,AA,McDonnell Douglas MD-82,4,MD82,BOEING,MD-82,2,Jet
168,American Airlines,AAL,AA,McDonnell Douglas MD-83,1,MD83,BOEING,MD-83,2,Jet
169,Delta Air Lines,DAL,DL,McDonnell Douglas MD-88,54,MD88,BOEING,MD-88,2,Jet


In [23]:
all_fleet_pd.drop(labels = ['model_no'], axis=1, inplace =True)
all_fleet_pd.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
all_fleet_pd

,Airline Name,ICAO ID,IATA ID,Aircraft Model,No Aircrafts,tdesig,manufacturer_code,engine_count,engine_type
0,American Airlines,AAL,AA,Airbus A319-100,133,A319,AIRBUS,2,Jet
3,Delta Air Lines,DAL,DL,Airbus A319-100,57,A319,AIRBUS,2,Jet
6,United Airlines,UAL,UA,Airbus A319-100,97,A319,AIRBUS,2,Jet
9,Spirit Airlines,NKS,NK,Airbus A319-100,31,A319,AIRBUS,2,Jet
12,Alaska Airlines,ASA,AS,Airbus A319-100,10,A319,AIRBUS,2,Jet
...,...,...,...,...,...,...,...,...,...
166,JetBlue Airways,JBU,B6,Embraer EMB-190,60,EMB1,NaN,NaN,NaN
167,American Airlines,AAL,AA,McDonnell Douglas MD-82,4,MD82,BOEING,2,Jet
168,American Airlines,AAL,AA,McDonnell Douglas MD-83,1,MD83,BOEING,2,Jet
169,Delta Air Lines,DAL,DL,McDonnell Douglas MD-88,54,MD88,BOEING,2,Jet


In [24]:
file_name = 'full_fleet' + today.strftime("%d%m%y") + '.csv'
all_fleet_pd.to_csv(file_name)

# New Data Source

In [25]:
import pandas as pd
req_col = ['CARRIER', 'CARRIER_NAME', 'OPERATING_STATUS', 'MANUFACTURER','MODEL', 'CAPACITY_IN_POUNDS']
BTS_fleet_pd = pd.read_csv('1053933928_T_F41SCHEDULE_B43.csv', usecols= req_col)
BTS_fleet_pd      

,CARRIER,CARRIER_NAME,OPERATING_STATUS,MANUFACTURER,MODEL,CAPACITY_IN_POUNDS
0,0WQ,Avjet Corporation,Y,Boeing,BBJ7377EG,171000
1,0WQ,Avjet Corporation,Y,Bombardier,GlobalExpress,96000
2,0WQ,Avjet Corporation,Y,Bombardier,GlobalExpress,99500
3,0WQ,Avjet Corporation,Y,Dassault-Falcon-2000EXEASy,DASSAULT-FALCON-2000,90700
4,0WQ,Avjet Corporation,Y,Gulfstream,GA-IV-IV,74600
...,...,...,...,...,...,...
7690,ZW,Air Wisconsin Airlines Corp,Y,Bombardier,CL65,12972
7691,ZW,Air Wisconsin Airlines Corp,Y,Bombardier,CL65,12972
7692,ZW,Air Wisconsin Airlines Corp,Y,Bombardier,CL65,12972
7693,ZW,Air Wisconsin Airlines Corp,Y,Bombardier,CL65,12972


In [26]:
BTS_fleetsize_pd = BTS_fleet_pd.groupby(BTS_fleet_pd.columns.tolist(),as_index=False).size()
BTS_fleetsize_pd.rename(columns={"size": "NO. AIRCRAFTS"}, inplace = True)
BTS_fleetsize_pd

,CARRIER,CARRIER_NAME,OPERATING_STATUS,MANUFACTURER,MODEL,CAPACITY_IN_POUNDS,NO. AIRCRAFTS
0,0WQ,Avjet Corporation,Y,Boeing,BBJ7377EG,171000,1
1,0WQ,Avjet Corporation,Y,Bombardier,GlobalExpress,96000,1
2,0WQ,Avjet Corporation,Y,Bombardier,GlobalExpress,99500,1
3,0WQ,Avjet Corporation,Y,Dassault-Falcon-2000EXEASy,DASSAULT-FALCON-2000,90700,1
4,0WQ,Avjet Corporation,Y,Gulfstream,G150,26100,1
...,...,...,...,...,...,...,...
844,YX,Republic Airline,Y,Embraer,ERJ-170-100LR,82012,5
845,YX,Republic Airline,Y,Embraer,ERJ-170-100SE,82012,43
846,YX,Republic Airline,Y,Embraer,ERJ-170-100SU,82012,14
847,YX,Republic Airline,Y,Embraer,ERJ-170-200LR,85517,160


In [27]:
import re
# airlines_list = ['American Airlines Inc.', 'Southwest', 'Delta', 'United', 'Spirit', 'Alaska', 'JetBlue', 'SkyWest', 'Frontier', 'Hawaiian']
# airlines_str = ''
# for airline in airlines_list:
#     airlines_str= airlines_str + airline +'|'
# airlines_str = airlines_str[:-1]

airlines_list ='Alaska Airlines Inc.', 'American Airlines Inc.', 'Delta Air Lines Inc.', 'Frontier Airlines Inc.','Hawaiian Airlines Inc.', 'JetBlue Airways', 'SkyWest Airlines Inc.', 'Southwest Airlines Co.', 'Spirit Air Lines', 'United Air Lines Inc.'
BTS_fleetsize_pd = BTS_fleetsize_pd[(BTS_fleetsize_pd.CARRIER_NAME.isin(airlines_list)) & (BTS_fleetsize_pd.OPERATING_STATUS == 'Y')]
#BTS_fleetsize_pd = BTS_fleetsize_pd[(BTS_fleetsize_pd.CARRIER_NAME.str.contains(airlines_str, regex=True)) & (BTS_fleetsize_pd.OPERATING_STATUS == 'Y')]
BTS_fleetsize_pd

,CARRIER,CARRIER_NAME,OPERATING_STATUS,MANUFACTURER,MODEL,CAPACITY_IN_POUNDS,NO. AIRCRAFTS
131,AA,American Airlines Inc.,Y,Airbus,A319-112,35000,54
132,AA,American Airlines Inc.,Y,Airbus,A319-112,36700,8
133,AA,American Airlines Inc.,Y,Airbus,A319-115,35600,32
134,AA,American Airlines Inc.,Y,Airbus,A319-132,36700,39
135,AA,American Airlines Inc.,Y,Airbus,A320-214,37200,22
...,...,...,...,...,...,...,...
815,WN,Southwest Airlines Co.,Y,THEBOEINGCO,B-737-7K9,36200,4
816,WN,Southwest Airlines Co.,Y,THEBOEINGCO,B-737-7L9,36200,5
817,WN,Southwest Airlines Co.,Y,THEBOEINGCO,B-737-7Q8,36200,19
818,WN,Southwest Airlines Co.,Y,THEBOEINGCO,B-737-8,48421,41


In [28]:
BTS_fleetsize_pd.groupby(['CARRIER_NAME'])[["NO. AIRCRAFTS"]].sum()

,NO. AIRCRAFTS
CARRIER_NAME,
Alaska Airlines Inc.,197
American Airlines Inc.,877
Delta Air Lines Inc.,750
Frontier Airlines Inc.,104
Hawaiian Airlines Inc.,59
JetBlue Airways,267
SkyWest Airlines Inc.,561
Southwest Airlines Co.,718
Spirit Air Lines,155


In [29]:
str = 'A310-112'

def tdesig_maker (make):
    
    critical_wrd = make

    if '-' in critical_wrd:
        broken_mod_ls = critical_wrd.split('-')
        if  '0' in broken_mod_ls[0]:
            tdesig = broken_mod_ls[0][0:3] + broken_mod_ls[1][0]
        else:
            lead_mod = broken_mod_ls[0][0:4]
            tdesig = lead_mod + broken_mod_ls[1][0:(4-len(lead_mod))]
    else:
        tdesig = critical_wrd[0:4]
    return tdesig

tdesig_maker(str)

'A311'

In [30]:
BTS_fleetsize_pd['tdesig'] = BTS_fleetsize_pd.apply(lambda row: tdesig_maker(row['MODEL']),axis=1)
BTS_fleetsize_pd

<ipython-input-30-bbfd3f46422f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BTS_fleetsize_pd['tdesig'] = BTS_fleetsize_pd.apply(lambda row: tdesig_maker(row['MODEL']),axis=1)


,CARRIER,CARRIER_NAME,OPERATING_STATUS,MANUFACTURER,MODEL,CAPACITY_IN_POUNDS,NO. AIRCRAFTS,tdesig
131,AA,American Airlines Inc.,Y,Airbus,A319-112,35000,54,A319
132,AA,American Airlines Inc.,Y,Airbus,A319-112,36700,8,A319
133,AA,American Airlines Inc.,Y,Airbus,A319-115,35600,32,A319
134,AA,American Airlines Inc.,Y,Airbus,A319-132,36700,39,A319
135,AA,American Airlines Inc.,Y,Airbus,A320-214,37200,22,A322
...,...,...,...,...,...,...,...,...
815,WN,Southwest Airlines Co.,Y,THEBOEINGCO,B-737-7K9,36200,4,B737
816,WN,Southwest Airlines Co.,Y,THEBOEINGCO,B-737-7L9,36200,5,B737
817,WN,Southwest Airlines Co.,Y,THEBOEINGCO,B-737-7Q8,36200,19,B737
818,WN,Southwest Airlines Co.,Y,THEBOEINGCO,B-737-8,48421,41,B737


In [31]:
all_fleet_pd = pd.merge(
    BTS_fleetsize_pd,
    type_des_db,
    how="left",
    on=None,
    left_on= 'tdesig',
    right_on= 'tdesig',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)
all_fleet_pd

,CARRIER,CARRIER_NAME,OPERATING_STATUS,MANUFACTURER,MODEL,CAPACITY_IN_POUNDS,NO. AIRCRAFTS,tdesig,manufacturer_code,model_no,engine_count,engine_type
0,F9,Frontier Airlines Inc.,Y,Airbus,319-111,141100,1,3191,NaN,NaN,NaN,NaN
1,F9,Frontier Airlines Inc.,Y,Airbus,319-112,141100,3,3191,NaN,NaN,NaN,NaN
2,F9,Frontier Airlines Inc.,Y,Airbus,320-114,162000,2,3201,NaN,NaN,NaN,NaN
3,F9,Frontier Airlines Inc.,Y,Airbus,320-211,162000,1,3202,NaN,NaN,NaN,NaN
4,F9,Frontier Airlines Inc.,Y,Airbus,320-214,162000,16,3202,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
254,OO,SkyWest Airlines Inc.,Y,Bombardier,CRJ-700,74957,146,CRJ7,BOMBARDIER,CL-600\t,2,Jet
255,OO,SkyWest Airlines Inc.,Y,Bombardier,CRJ-900,75000,43,CRJ9,NaN,NaN,NaN,NaN
256,B6,JetBlue Airways,Y,Embraer,E190-100,25200,16,E191,NaN,NaN,NaN,NaN
257,B6,JetBlue Airways,Y,Embraer,E190-100,26200,44,E191,NaN,NaN,NaN,NaN


In [ ]:
import re
man_ls = []

for model in fleet_pd['Aircraft Model']:
    mod_words = re.split(' |-',model)
    first_word = mod_words[0].upper()
    if first_word.upper() not in man_ls:
        man_ls.append(first_word)
print(man_ls)